# Imports and setup

In [98]:
# Basic
import numpy as np
import scipy
import scipy.stats
import os
import itertools
import warnings
import sys
from copy import deepcopy

# Data Loading
import cmlreaders as cml #Penn Computational Memory Lab's library of data loading functions

# Data Handling
import os
from os import listdir as ld
import os.path as op
from os.path import join, exists as ex
import time
import datetime

# Data Analysis
import pandas as pd
import xarray as xr

# EEG & Signal Processing
import ptsa
from ptsa.data.readers import BaseEventReader, EEGReader, CMLEventReader, TalReader
from ptsa.data.filters import MonopolarToBipolarMapper, MorletWaveletFilter
from ptsa.data.timeseries import TimeSeries

# Data Visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Parallelization
import cmldask.CMLDask as da
from cmldask.CMLDask import new_dask_client_slurm as cl
from cluster import wait, get_exceptions_quiet as get_ex
import cmldask

# Custom
from cstat import * #circular statistics
from misc import * #helper functions for loading and saving data, and for other purposes
from matrix_operations import * #matrix operations

from helper import *

%load_ext autoreload

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

beh_to_event_windows = {'en': [250-1000, 1250+1000],
                     'en_all': [250-1000, 1250+1000],
                     'rm': [-1000, 0],
                     'ri': [-1000, 0]}

beh_to_epochs = {'en': np.arange(250, 1250, 200),
              'en_all': np.arange(250, 1250, 200),
              'rm': np.arange(-1000, 0, 200),
              'ri': np.arange(-1000, 0, 200)}


from helper import root_dir, USERNAME as user
if not os.path.exists(root_dir):
    os.mkdir(root_dir)

from functools import partial
cluster_log_dir = 'cluster'
cl = partial(cl, log_directory=cluster_log_dir)
if not os.path.exists(cluster_log_dir):
    os.mkdir(cluster_log_dir)

font_dirs = ['fonts']


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Compare dataset between initial submission and new submission

In [30]:
old_sess_list_df = pd.read_json(join(root_dir, 'initial_submission', 'sess_list_df.json')).query('include == True')[['sub', 'exp', 'sess', 'loc', 'mon']]
old_sess_list_df_ri = pd.read_json(join(root_dir, 'initial_submission', 'sess_list_df.json')).query('(include == True) & (ri_events == True)')[['sub', 'exp', 'sess', 'loc', 'mon']]

In [31]:
sess_list_df = pd.read_json(join(root_dir, 'sess_list_df.json')).query('include == True')[['sub', 'exp', 'sess', 'loc', 'mon']]
sess_list_df_ri = pd.read_json(join(root_dir, 'sess_list_df.json')).query('(include == True) & (ri_events == True)')[['sub', 'exp', 'sess', 'loc', 'mon']]

In [32]:
print(f'{old_sess_list_df.shape[0]} sessions analyzed in main analysis in initial submission.')
print(f'{sess_list_df.shape[0]} sessions analyzed in main analysis in new submission.')
print(f'{old_sess_list_df_ri.shape[0]} sessions analyzed in recall accuracy analysis in initial submission.')
print(f'{sess_list_df_ri.shape[0]} sessions analyzed in recall accuracy analysis in new submission.')

1071 sessions analyzed in main analysis in initial submission.
980 sessions analyzed in main analysis in new submission.
365 sessions analyzed in recall accuracy analysis in initial submission.
317 sessions analyzed in recall accuracy analysis in new submission.


In [33]:
print(f'{old_sess_list_df["sub"].unique().shape[0]} subjects analyzed in main analysis in initial submission.')
print(f'{sess_list_df["sub"].unique().shape[0]} subjects analyzed in main analysis in new submission.')
print(f'{old_sess_list_df_ri["sub"].unique().shape[0]} subjects analyzed in recall accuracy analysis in initial submission.')
print(f'{sess_list_df_ri["sub"].unique().shape[0]} subjects analyzed in recall accuracy analysis in new submission.')

413 subjects analyzed in main analysis in initial submission.
378 subjects analyzed in main analysis in new submission.
194 subjects analyzed in recall accuracy analysis in initial submission.
167 subjects analyzed in recall accuracy analysis in new submission.


# Calculate number of repeated recalls initially analyzed

In [40]:
old_sess_list_df = pd.read_json(join(root_dir, 'initial_submission', 'sess_list_df.json')).query('include == True')[['sub', 'exp', 'sess', 'loc', 'mon', 'evs_data_source']]
ev_count = 0
repeat_count = 0
for iSess, rSess in old_sess_list_df.iterrows():
    dfrow = rSess[['sub', 'exp', 'sess', 'loc', 'mon']]
    evs = pd.read_json(join(root_dir, 'initial_submission', 'rm', 'events', f'{ftag(dfrow)}_events.json'))
    matcher = match_events.MatchedEvents(dfrow, get_sr(dfrow), rSess['evs_data_source'])
    for iEv, ev in evs.query('type=="REC_WORD"').iterrows():
        ev_count += 1
        m = ev['mstime']
        lookup = matcher.all_recs[(matcher.all_recs['mstime'] == m)]
        assert len(lookup) == 1
        lookup = lookup.iloc[0]
        if lookup['repeat'] == 1: repeat_count += 1

/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:80: UserWarning: Multiple eegfile values, sorting events by mstime.
  warnings.warn("Multiple eegfile values, sorting events by mstime.")
/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:80: UserWarning: Multiple eegfile values, sorting events by mstime.
  warnings.warn("Multiple eegfile values, sorting events by mstime.")
/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:80: UserWarning: Multiple eegfile values, sorting events by mstime.
  warnings.warn("Multiple eegfile values, sorting events by mstime.")
/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:80: UserWarning: Multiple eegfile values, sorting events by mstime.
  warnings.warn("Multiple eegfile values, sorting events by mstime.")


In [41]:
print_header('rm')
print(f'{ev_count} total events analyzed.\n\ {repeat_count} repeats. {(repeat_count/ev_count*100):.2}% of events were repeats.')

---------rm---------
26760 total events analyzed.
\ 1377 repeats. 5.1% of events were repeats.


In [43]:
old_sess_list_df = pd.read_json(join(root_dir, 'initial_submission', 'sess_list_df.json')).query('(include == True) & (ri_events == True)')[['sub', 'exp', 'sess', 'loc', 'mon', 'evs_data_source']]
ev_count = 0
repeat_count = 0
for iSess, rSess in old_sess_list_df.iterrows():
    dfrow = rSess[['sub', 'exp', 'sess', 'loc', 'mon']]
    evs = pd.read_json(join(root_dir, 'initial_submission', 'ri', 'events', f'{ftag(dfrow)}_events.json'))
    matcher = match_events.MatchedEvents(dfrow, get_sr(dfrow), rSess['evs_data_source'])
    for iEv, ev in evs.query('type=="REC_WORD"').iterrows():
        ev_count += 1
        m = ev['mstime']
        lookup = matcher.all_recs[(matcher.all_recs['mstime'] == m)]
        assert len(lookup) == 1
        lookup = lookup.iloc[0]
        if lookup['repeat'] == 1: repeat_count += 1

/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:80: UserWarning: Multiple eegfile values, sorting events by mstime.
  warnings.warn("Multiple eegfile values, sorting events by mstime.")
/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:80: UserWarning: Multiple eegfile values, sorting events by mstime.
  warnings.warn("Multiple eegfile values, sorting events by mstime.")
/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:80: UserWarning: Multiple eegfile values, sorting events by mstime.
  warnings.warn("Multiple eegfile values, sorting events by mstime.")
/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:80: UserWarning: Multiple eegfile values, sorting events by mstime.
  warnings.warn("Multiple eegfile values, sorting events by mstime.")


In [44]:
print_header('ri')
print(f'{ev_count} total events analyzed.\n\ {repeat_count} repeats. {(repeat_count/ev_count*100):.2}% of events were repeats.')

---------ri---------
11846 total events analyzed.
\ 869 repeats. 7.3% of events were repeats.


# Check duplicate sessions

In [2]:
import pandas as pd
import numpy as np
import xarray as xr
from cmlreaders import CMLReader, PathFinder
import hashlib

# returns cmlreader from experiment, subject, session (without need for localization/montage)
# TODO exclude pyFR sessions or require localization/montage to ensure unique sessions?
def get_reader(row_df=None, *, experiment=None, subject=None, session=None, index_df=None, return_finder=False):
    if isinstance(row_df, type(None)):
        assert not (isinstance(experiment, type(None)) or isinstance(subject, type(None)) or isinstance(session, type(None)))
    else:
        assert isinstance(experiment, type(None)) and isinstance(subject, type(None)) and isinstance(session, type(None))
        if isinstance(row_df, pd.DataFrame):
            if len(row_df) != 1:
                raise ValueError('row_df must have length 1!')
            row_df = row_df.iloc[0]
        experiment = row_df.experiment
        subject = row_df.subject
        session = row_df.session
    if isinstance(index_df, type(None)):
        from cmlreaders import get_data_index
        index_df = get_data_index()
    row = index_df.query('experiment == @experiment and subject == @subject and session == @session')
    if len(row) == 0: raise ValueError(f'{subject} {experiment} session {session} not found!')
    elif len(row) != 1: raise ValueError(f'{subject} {experiment} session {session} not unique!')
    CMLClass = PathFinder if return_finder else CMLReader
    return CMLClass(experiment=experiment, subject=subject, session=session,
                         localization=row.localization.item(), montage=row.montage.item())


def hash_data_structure(data):
    # Convert the data structure to a string representation
    if isinstance(data, list):
        data_str = str(data)
    elif isinstance(data, np.ndarray):
        data_str = np.array2string(data)
    elif isinstance(data, xr.DataArray):
        # Convert xarray DataArray to a NumPy array string representation
        data_str = np.array2string(data.values)
    elif isinstance(data, pd.DataFrame):
        data_str = data.to_string()
    else:
        try:
            # Attempt to create a string representation using repr
            data_str = repr(data)
        except Exception as e:
            raise ValueError("Unsupported data type")

    # Create a hash of the string representation
    return hashlib.sha256(data_str.encode()).hexdigest()


def find_duplicate_sessions(events, duplicate_ratio=0.05, check_eeg=True, 
                            subject_column='subject',
                            experiment_column='experiment',
                            session_column='session',
                            check_columns=['subject', 'mstime', 'type'],
                            return_events_without_duplicate_sessions=True, verbose=True):
    assert (duplicate_ratio > 0) and (duplicate_ratio <= 1)
    session_columns = [subject_column, experiment_column, session_column]
    duplicate_events = events.copy()
    duplicate_events['duplicate'] = duplicate_events.duplicated(check_columns, keep=False)
    duplicate_sessions = duplicate_events[session_columns + ['duplicate']].groupby(session_columns).mean()\
            .reset_index().drop_duplicates(session_columns).query('duplicate > @duplicate_ratio').reset_index()
    if duplicate_sessions.empty:
        if return_events_without_duplicate_sessions:
            return events, duplicate_sessions
        return duplicate_sessions
    
    duplicate_events = duplicate_events.merge(duplicate_sessions[session_columns], on=session_columns)
    index_df = get_data_index()
    
    duplicate_sessions['keep'] = duplicate_sessions.duplicate <= duplicate_ratio
    if check_eeg:
        # confirm EEG duplicated between all pairs of duplicate sessions
        # exclude duplicated sessions without matching EEG
        duplicate_sessions['eeg_hash'] = ''
        for sess_idx, sess_df in duplicate_sessions[session_columns].iterrows():
            reader = get_reader(sess_df, index_df=index_df)
            eeg = reader.load_eeg()
            duplicate_sessions.loc[sess_idx, ['eeg_hash']] = hash_data_structure(eeg.data)
            del eeg
        
        # display(duplicate_sessions)
        for eeg_hash, sessions in duplicate_sessions.groupby(['eeg_hash']):
            if (len(sessions[subject_column].unique()) > 1) or (len(sessions[experiment_column].unique()) > 1):
                print('WARNING: EEG matches between sessions with different subjects/experiments! Dropping sessions:')
                display(sessions)
            elif len(sessions) < 2:
                print('WARNING: EEG from session with duplicated events does not match '
                      'EEG of any other duplicated sessions! Dropping session:')
                display(sessions)
            else:
                keep_session = sessions.iloc[0:1]
                duplicate_sessions.loc[keep_session.index, ['keep']] = True
    else:
        raise NotImplementedError
        # for eeg_hash, sessions in duplicate_sessions.groupby([subject_column, experiment_column]):
        #     keep_session = sessions.iloc[0:1]
        #     keep_sessions.append(keep_session)
    
    if verbose and not duplicate_sessions.empty:
        print('Duplicate sessions:')
        display(duplicate_sessions)
    
    if return_events_without_duplicate_sessions:
        events = events.merge(duplicate_sessions.query('not keep')[session_columns], 
                              on=session_columns, how='left', indicator=True)
        events = events.query('_merge == "left_only"').drop(columns=['_merge'])
        return events, duplicate_sessions
    
    return duplicate_sessions

In [3]:
def load_raw_events(dfrow):
    
    sub, exp, sess, loc, mon = dfrow[['sub', 'exp', 'sess', 'loc', 'mon']]
    if dfrow['evs_data_source'] == 'cmlreaders':
        reader = cml.CMLReader(*dfrow[['sub', 'exp', 'sess', 'loc', 'mon']])
        events = reader.load('events')
        if not exp == 'pyFR':
            events = cml.correct_retrieval_offsets(events, reader)
            events = cml.sort_eegfiles(events)
    elif dfrow['evs_data_source'] == 'ptsa':
        exp_dict = {'FR1': 'RAM_FR1', 'catFR1': 'RAM_CatFR1', 'pyFR': 'pyFR'}
        exp_ = exp_dict[exp]
        mon_ = '' if mon ==0 else f'_{mon}' #for tal_reader path name
        events = BaseEventReader(filename=f'/data/events/{exp_}/{sub}{mon_}_events.mat', use_reref_eeg=False).read()
        exclude_cols = ['stimParams'] if 'stimParams' in events.dtype.names else []
        events = pd.DataFrame.from_records(events, exclude=exclude_cols)
        events = events[events.session==sess]
        events = events.sort_values(by='mstime')
        # retrieval offset corrections are not necessary for pyFR sessions, and only pyFR sessions need to be loaded through the ptsa readers

    for col, value in zip(['experiment', 'protocol', 'montage'],
                          [exp, np.nan, mon]):
        if col not in events.columns:
            events.insert(len(events.columns), col, [value]*len(events))

    events.rename(columns={'list': 'trial'}, inplace=True)

    #session-specific changes
    dfrow_key = tuple(dfrow)
    errors = {('R1171M', 'FR1', 2, 0, 0): 'mstime < 1462393964709 & mstime > 1462394113265',
              ('R1329T', 'FR1', 0, 0, 0): 'trial != 4',
              ('R1341T', 'FR1', 1, 0, 0): 'trial != 8',
              ('R1374T', 'FR1', 0, 0, 0): 'trial != 1',
              ('R1488T', 'catFR1', 0, 0, 0): 'trial != 11',
              ('TJ040', 'pyFR', 0, 0, 1): 'trial != 6',
              ('FR060', 'pyFR', 1, 0, 0): 'trial != 1'}

    if dfrow_key in errors.keys(): events = events.query(errors[dfrow_key])
    
    return events

In [4]:
sldf=pd.read_json(join(root_dir, 'sess_list_df.json'))[['sub', 'exp', 'sess', 'loc', 'mon', 'evs_data_source']].query('sub=="R1408N"')

In [5]:
# sess_list_df = pd.read_json(join(root_dir, 'sess_list_df.json'))[['sub', 'exp', 'sess', 'loc', 'mon', 'evs_data_source']]
all_events = []
for iDfrow, dfrow in sldf.iterrows():
    events = load_raw_events(dfrow)
    events['duplicated_within_session'] = events.duplicated(['mstime', 'type'], keep=False)
    if np.sum(events['duplicated_within_session']) > 0:
        print(dfrow)
    all_events.append(events)
    
    
    

/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:57: UserWarning: Applying 1000 ms offset correction to retrieval events.
  warnings.warn(f'Applying {ms} ms offset correction to retrieval events.')
/usr/global/miniconda/py310_23.1.0-1/envs/workshop_311/lib/python3.11/site-packages/cmlreaders/event_process.py:57: UserWarning: Applying 1000 ms offset correction to retrieval events.
  warnings.warn(f'Applying {ms} ms offset correction to retrieval events.')


In [6]:
all_events = pd.concat(all_events,axis=0)

In [9]:
all_events.query('duplicated_within_session == True')[['eegoffset', 'eegfile', 'item_name', 'type', 'mstime']]

,eegoffset,eegfile,item_name,type,mstime
0,14179,R1171M_catFR1_0_04May16_1447,X,INSTRUCT_VIDEO,1462373256412
1,14179,R1171M_catFR1_0_04May16_1447,X,INSTRUCT_VIDEO,1462373256412
0,6125,R1230J_FR1_2_12Oct16_1435,X,INSTRUCT_VIDEO,1476283063550
1,6125,R1230J_FR1_2_12Oct16_1435,X,INSTRUCT_VIDEO,1476283063550
0,812953,R1230J_FR1_5_14Oct16_1402,X,INSTRUCT_VIDEO,1476454668979
1,812953,R1230J_FR1_5_14Oct16_1402,X,INSTRUCT_VIDEO,1476454668979
0,6330,R1231M_FR1_0_10Oct16_1939,X,INSTRUCT_VIDEO,1476128503220
1,6330,R1231M_FR1_0_10Oct16_1939,X,INSTRUCT_VIDEO,1476128503220
0,5473,R1231M_FR1_1_12Oct16_1913,X,INSTRUCT_VIDEO,1476299770372
1,5473,R1231M_FR1_1_12Oct16_1913,X,INSTRUCT_VIDEO,1476299770372


In [21]:
cml.get_data_index().query('subject=="R1486J"')

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
2251,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,LocationSearch,build,0,NaN,0,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,2,R1486J,R1486J,3.4,protocols/r1/subjects/R1486J/experiments/Locat...
2252,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,LocationSearch,build,0,NaN,0,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,3,R1486J,R1486J,3.4,protocols/r1/subjects/R1486J/experiments/Locat...
2253,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,LocationSearch,build,0,NaN,1,NaN,2,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,4,R1486J,R1486J_1,3.4,protocols/r1/subjects/R1486J/experiments/Locat...
2254,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,LocationSearch,build,0,NaN,1,NaN,3,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,5,R1486J,R1486J_1,3.4,protocols/r1/subjects/R1486J/experiments/Locat...
2255,NaN,protocols/r1/subjects/R1486J/experiments/catFR...,protocols/r1/subjects/R1486J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1486J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,0,R1486J,R1486J,3.4,protocols/r1/subjects/R1486J/experiments/catFR...
2256,NaN,protocols/r1/subjects/R1486J/experiments/catFR...,protocols/r1/subjects/R1486J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1486J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,1,R1486J,R1486J,3.4,protocols/r1/subjects/R1486J/experiments/catFR...
2257,NaN,protocols/r1/subjects/R1486J/experiments/catFR...,protocols/r1/subjects/R1486J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1486J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,2,R1486J,R1486J,3.4,protocols/r1/subjects/R1486J/experiments/catFR...
2258,NaN,protocols/r1/subjects/R1486J/experiments/catFR...,protocols/r1/subjects/R1486J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1486J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,3,R1486J,R1486J,3.4,protocols/r1/subjects/R1486J/experiments/catFR...
2259,NaN,protocols/r1/subjects/R1486J/experiments/catFR...,protocols/r1/subjects/R1486J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1486J/experiments/catFR...,1,NaN,0,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,4,R1486J,R1486J_1,3.4,protocols/r1/subjects/R1486J/experiments/catFR...
2260,NaN,protocols/r1/subjects/R1486J/experiments/catFR...,protocols/r1/subjects/R1486J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1486J/experiments/catFR...,1,NaN,1,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,5,R1486J,R1486J_1,3.4,protocols/r1/subjects/R1486J/experiments/catFR...


In [7]:
from cmlreaders import get_data_index
events, duplicate_sessions = find_duplicate_sessions(all_events)

,index,subject,experiment,session,duplicate,keep,eeg_hash
1,1,R1408N,catFR1,1,0.093949,False,1f439ba4e855e5d923a7187930014d78a14a8fc300d283...


,index,subject,experiment,session,duplicate,keep,eeg_hash
0,0,R1408N,catFR1,0,0.108059,False,e412c353bfae308876960d8b156416bc396a714d5d41c7...


Duplicate sessions:


,index,subject,experiment,session,duplicate,keep,eeg_hash
0,0,R1408N,catFR1,0,0.108059,False,e412c353bfae308876960d8b156416bc396a714d5d41c7...
1,1,R1408N,catFR1,1,0.093949,False,1f439ba4e855e5d923a7187930014d78a14a8fc300d283...


In [12]:
a = cml.CMLReader(*('R1408N', 'catFR1', 1, 0, 0)).load('events')['mstime']

In [25]:
all_events['duplicated'] = all_events.duplicated(['mstime', 'type'], keep=False)

In [27]:
e=all_events.query('(duplicated==True) & (duplicated_within_session==False)')[['eegoffset', 'eegfile', 'mstime', 'type', 'session']]

In [32]:
e['type'].unique()

array(['PROB', 'START', 'STOP'], dtype=object)

In [20]:
all_events.query('session==0')['mstime'].max()

1522527493009

In [21]:
all_events.query('session==1')['mstime'].min()

1522526060000

# Check encoding

In [3]:
old_sess_list_df = pd.read_json(join(root_dir, 'initial_submission', 'sess_list_df.json')).query('include == True')[['sub', 'exp', 'sess', 'loc', 'mon']]
sess_list_df = pd.read_json(join(root_dir, 'sess_list_df.json')).query('include == True')[['sub', 'exp', 'sess', 'loc', 'mon']]


In [26]:
def get_no_events(dfrow):
    return len(pd.read_json(join(root_dir, 'en', 'events', f'{ftag(dfrow)}_events.json')))
def get_no_events_old(dfrow):
    return len(pd.read_json(join(root_dir, 'initial_submission', 'en', 'events', f'{ftag(dfrow)}_events.json')))

In [24]:
sess_list_df['no_en_events'] = sess_list_df.apply(lambda r: get_no_events(r), axis=1)

In [27]:
sess_list_df['no_en_events_old'] = sess_list_df.apply(lambda r: get_no_events_old(r), axis=1)

In [29]:
sess_list_df

,sub,exp,sess,loc,mon,no_en_events,no_en_events_old
"('R1001P', 'FR1', 0, 0, 0)",R1001P,FR1,0,0,0,106,106
"('R1001P', 'FR1', 1, 0, 0)",R1001P,FR1,1,0,0,124,124
"('R1002P', 'FR1', 0, 0, 0)",R1002P,FR1,0,0,0,214,214
"('R1002P', 'FR1', 1, 0, 0)",R1002P,FR1,1,0,0,216,216
"('R1003P', 'FR1', 0, 0, 0)",R1003P,FR1,0,0,0,160,160
...,...,...,...,...,...,...,...
"('UP041', 'pyFR', 0, 0, 0)",UP041,pyFR,0,0,0,88,88
"('UP041', 'pyFR', 1, 0, 0)",UP041,pyFR,1,0,0,80,80
"('UP041', 'pyFR', 2, 0, 0)",UP041,pyFR,2,0,0,56,56
"('UP045', 'pyFR', 1, 0, 0)",UP045,pyFR,1,0,0,86,86
